# 3.1 지도학습

### 3.1.1 K-최근접 이웃(K-nearest neighbor)
새로운 입력을 받았을 때, 기존 클러스터에서 모든 데이터와 인스턴스 기반 거리를 측정해서 가장 많은 속성을 갖는 클러스터에 할당하는 분류 알고리즘
- 기존 데이터들과 하나씩 거리를 계산하고 거리상 가장 가까운 데이터 k개를 선택하여 해당 클러스터에 할당
- k값에 따라 분류결과가 달라짐

In [3]:
# 라이브러리 호출 및 데이터 준비
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics 

names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

dataset = pd.read_csv('data/chap3/data/iris.data', names=names)

In [5]:
dataset

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [20]:
# 훈련과 검증 데이터셋 분리
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

In [21]:
# 모델 생성 및 훈련
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=50)

In [22]:
# 모델 정확도
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_test)
print("정확도: {}".format(accuracy_score(y_test, y_pred)))

정확도: 0.8666666666666667


In [23]:
# 최적 K 찾기
k = 10
acc_array = np.zeros(k)
# k값 돌려가면서 정확도 저장
for k in np.arange(1, k+1, 1):
    classifier = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    acc_array[k-1] = acc
    
print(acc_array)
# 최대값 산출    
max_acc = np.amax(acc_array)
acc_list = list(acc_array)
k = acc_list.index(max_acc)
print("정확도", max_acc, "으로 최적의 k는", k+1, "입니다.")

[0.9 1.  1.  1.  1.  1.  1.  1.  1.  1. ]
정확도 1.0 으로 최적의 k는 2 입니다.


---
### 3.1.2 서포트 벡터 머신 (Supprot Vector Machine, SVM)
분류를 위한 기준선을 정의하는 모델로